<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Notebook: Database Connectivity Testing and Configuration for Watson OpenScale

This notebook helps you test and update the database connection between your PostgreSQL instance and Watson OpenScale.

## ⚠️ IMPORTANT WARNINGS

**DANGER - READ BEFORE PROCEEDING:**
- **The last cell (database configuration replacement) is NOT REVERSIBLE**
- Once executed, you can rerun it to patch forcefully, but **we do not guarantee any recovery or rollback**
- **CRITICAL REQUIREMENT:** You MUST use **identical schema names** between your old and new PostgreSQL instances
- If schema names don't match, Watson OpenScale will not be able to find your data tables and monitoring will fail
- **Backup your current configuration before proceeding**

**Prerequisites:**
- IBM Cloud account with Watson OpenScale service
- PostgreSQL database instance on IBM Cloud with **identical schema names** to your existing instance
- API key for Watson OpenScale
- Database service credentials

**What this notebook does:**
1. Tests connectivity between your PostgreSQL database and Watson OpenScale
2. Verifies that database schemas are accessible
3. **IRREVERSIBLY** updates the database configuration in your Watson OpenScale data mart

**Contents:**
1. [Package Installation](#Package-installation)
2. [Setting up Services](#Setting-up-Services)
3. [Steps to create service credentials](#Steps-to-create-service-credentials)
4. [Getting current database configuration](#Getting-current-database-configuration)
5. [Replace database configuration with the new one](#Replace-database-configuration-with-the-new-one)

## Package installation 
You can skip this installation, depending on your environment

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

Action: restart the kernel if you executed above cell!

## Setting up Services

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient

service_credentials = {
    "url": "https://aiopenscale.cloud.ibm.com", # replace this url, depending on your working env
    "apikey": "REPLACE YOUR API KEY"
}

authenticator = IAMAuthenticator(
    apikey=service_credentials['apikey'],
)

client = APIClient(
    service_url=service_credentials['url'],
    authenticator=authenticator
)

print(client.version)

[Warning] No region provided : Using default region as us-south
3.1.3


## Create service credentials of PostgreSQL

Follow this documentation, https://cloud.ibm.com/docs/account?topic=account-service_credentials&interface=ui, and create a service credential for PostgreSQL.

Once you created it, please copy it as JSON, and paste it to `database_credentials` variable, without any modification.

Of course, you can reuse existing credentials to define this variables

In [ ]:
# Please replace the following with your own credentials

database_credentials = {
  "connection": {
    "cli": {
      "arguments": []
    }
  }
}


In [ ]:
body = {
    "database_type": "postgresql",
    "credentials": database_credentials
}

prepared_request = client.prepare_request("POST", "/v2/database_schemas", data=body)
result = client.send(prepared_request).get_result()
result

## result should list schemas in the database
## if you can see some schemas, as is in the example, then you are good to go, since the connection between your database and our service is established as expected

{'database_schemas': [{'compatible': True, 'name': 'ibm_extension'},
  {'compatible': True, 'name': 'information_schema'},
  {'compatible': True, 'name': 'pg_catalog'},
  {'compatible': True, 'name': 'public'},
  {'compatible': True, 'name': 'tiger'},
  {'compatible': True, 'name': 'tiger_data'}]}

## Getting current database configuration

In [ ]:
dataMart = client.data_marts.list(limit = 1)
secretId = dataMart.result.data_marts[0].entity.database_configuration.credentials.secret_id
print(secretId)

SECRET_URL = "/v2/secrets/{}".format(secretId)
prepared_request = client.prepare_request("GET", SECRET_URL)
result = client.send(prepared_request).get_result()
result

### Replace database configuration with the new one

In [ ]:
import json

patch_document = [
    {
        "op": "replace",
        "path": "/credentials",
        "value": database_credentials
    }
]

data = json.dumps(patch_document)
print(data)
prepared_request = client.prepare_request(method="PATCH", url=SECRET_URL,  data=data)
result = client.send(prepared_request).get_result()
result